# Linked data en JSON-LD

## Inleiding

In de voorgaande hoofdstukken heb je gezien hoe je met een schema ervoor kunt zorgen dat de documenten in een MongoDB database-collection gelijksoortig zijn.
In MongoDB kun je een query als schema koppelen aan een collection, om een document te valideren voordat je dit toevoegt of verandert.

In plaats van een MongoDB-query kun je ook de JSON-schema notatie gebruiken.
Hiermee kun je nog preciezer beschrijven wat de structuur van een document moet zijn, welke (veld)namen gebruikt moeten worden, en hoe de waarden getypeerd moeten zijn.

Door het gebruik van standaard-schema's zoals die van http://schema.org kun je ervoor zorgen dat de betekenis van de velden duidelijk vastligt, in elk geval voor de menselijke lezer. 
En je zorgt ervoor dat je documenten uitwisselbaar zijn met anderen die dezelfde schema's gebruiken.

Een volgende vraag is hoe je de eigenschappen van een schema expliciet kunt maken, zodat deze ook gebruikt (geïnterpreteerd) kunnen worden door computers, en niet alleen door mensen. Hiervoor kun je gebruik maken van de Linked Data technologie zoals die ontwikkeld is voor het web.

* hoe kun je duidelijk maken welke standaard-schema's je gebruikt?
* hoe kun je het verband met andere data aangeven?


## Linked data

Volgens Tim Berners-Lee, de uitvinder van het web, bestaat Linked Data uit de volgende stappen:
    
zie: https://www.w3.org/DesignIssues/LinkedData.html

1. Use URIs as names for things
2. Use HTTP URIs so that people can look up those names.
3. When someone looks up a URI, provide useful information, using the standards (RDF*, SPARQL)
4. Include links to other URIs. so that they can discover more things.    

## (1) Gebruik URI's/IRI's als namen

De eerste stap is het gebruiken van globale unieke identificaties voor "dingen" (resources).
Dergelijke globale identificaties zijn iet afhankelijk van de lokale context:
Dit betekent dat je gegevens gemakkelijk kunt uitwisselen tussen verschillende lokale contexten.
In het web is voor deze globale identificatie de URI (tegenwoordig: IRI) ontwikkeld.

> De web-term voor "ding" is "resource": dit kan iets concreets zijn: een ding, mens, dier, planeet; het kan ook iets virtueels of abstracts zijn. Het hoeft niet in het internet/web aanwezig te zijn, om het in het web te kunnen beschrijven.

Enkele voorbeelden van URI's/IRI's:

* urn:isbn:0-486-27557-4  (een ISBN-nummer identificeert een boek-editie)
* urn:dev:mac:0024befffe804ff1 (een MAC-adres identificeert een apparaat/netwerk-aansluiting)
* tel:+1-816-555-1212
* mailto:John.Doe@example.com
* ftp://ftp.is.co.za/rfc/rfc1808.txt
* http://www.ietf.org/rfc/rfc2396.txt

De eerste van deze voorbeelden zijn identificaties die je niet als link in het web kunt gebruiken.
Deze vallen onder de categorie "Uniform Resource Numbers" (urn).
De laatste voorbeelden, vanaf het telefoonnummer, kun je wel als link in een webpagina gebruiken.
De browser kan deze identificaties interpreteren, en er actie op ondernemen.
Bij de laatste twee gevallen kun je via de browser meer gegevens bij de identificatie ophalen.

> Het belangrijkste verschil tussen URI's en IRI's is dat deze laatste veel meer mogelijkheden bieden om een alternatief alfabet te gebruiken. URI's zijn traditioneel sterk gericht op het latijnse alfabet (ASCII).

* https://tools.ietf.org/html/rfc3986 (internet-RFC over URI's)
* https://tools.ietf.org/html/rfc3987 (IRI's)
* https://en.wikipedia.org/wiki/Internationalized_Resource_Identifier

## (2) Gebruik URL's als URI's

De tweede stap is: gebruik waar mogelijk URL's als URI's.
Een URL kun je als link in het web gebruiken, en via een URL kun je meer gegevens bij de identificatie vinden (zie de volgende stap).

Je kunt dit toepassen op de veldnamen van een document.
Als je de namen gebruiken van schema.org krijgen je bijvoorbeeld:

```
{
    "http://schema.org/name": "Harry van Doorn",
    "http://schema.org/email": "harryvdoorm@friendmail.org",
    "http://schema.org/telephone": "+31-6-1357 8642"
}
```

Dit is voor computers duidelijk en ondubbelzinnig, maar voor mensen wat omslachtig.
Met behulp van JSON-LD kun je dit compacter en overzichtelijker beschrijven.

In [157]:
import requests
import json
from bs4 import BeautifulSoup
from pyld import jsonld



## JSON-LD

In JSON-LD kun je bij een JSON-object aangeven welke context gebruikt wordt.
Anders gezegd: waar je de definitie van de gebruikte namen kunt vinden.
Het bovenstaande voorbeeld wordt dan:

```
{
  "@context: "http://schema.org",
  "name": "Harry van Doorn",
  "email": "harryvdoorm@friendmail.org",
  "telephone": "+31-6-1357 8642"
}
```

Met behulp van de hulpprogramma's bij JSON-LD (uit pyld) kun je dit expanderen.

> Door een fout in de python-implementatie moet je hier in plaats van "http://schmema.org" gebruiken: "http://schema.org/docs/jsonldcontext.jsonld". In een volgende versie is dat waarschijnlijk opgelost.

In [171]:
mycontact = {
  "@context": "http://schema.org/docs/jsonldcontext.jsonld",
  "name": "Harry van Doorn",
  "email": "harryvdoorm@friendmail.org",
  "tel": "+31-6-1357 8642"
}

expanded_contact = jsonld.expand(mycontact)
expanded_contact

[{'http://schema.org/email': [{'@value': 'harryvdoorm@friendmail.org'}],
  'http://schema.org/name': [{'@value': 'Harry van Doorn'}],
  'http://schema.org/tel': [{'@value': '+31-6-1357 8642'}]}]

Het omgekeerde is ook mogelijk: je kunt uit een geëxpandeerd document en een context weer een compact document maken.

> De volgorde van de velden in het document kan hierbij veranderen, maar dat heeft geen betekenis, noch in JSON, noch in Python.

In [172]:
jsonld.compact(expanded_contact, "http://schema.org/docs/jsonldcontext.jsonld")

{'@context': 'http://schema.org/docs/jsonldcontext.jsonld',
 'email': 'harryvdoorm@friendmail.org',
 'name': 'Harry van Doorn',
 'tel': '+31-6-1357 8642'}

**Opdrachten**

* Ga na wat er gebeurt als je een naam gebruikt die niet in schema.org gedefinieerd wordt, bijvoorbeeld "tel" in plaats van "telephone".
** welke URL krijg je als veldnaam?
** wat gebeurt er als je die URL gebruikt in de browser? (als je op die URL klikt?, anders gezegd: als je het document bij die URL ophaalt?)
* Vraag de lijst met namen in de browser op, via de link http://schema.org/docs/jsonldcontext.jsonld

### Links

* https://json-ld.org
* https://json-ld.org/playground/

## (3) Geef extra informatie via de URL's

De volgende stap is om via de URL's die als naam gebruikt worden, meer informatie over de naam en het bijbehorende "ding" te geven.

Welke gegevens krijg je bij de URL "[http://schema.org/email](http://schema.org/email)"?

> We gebruiken hier de Python-library `requests`, voor het versturen van HTTP-requests vanuit Python. (zie: https://requests.readthedocs.io/)

In [173]:
r = requests.get("http://schema.org/email")
r.status_code

200

Als de status_code = 200 ("OK"), dan was de aanvraag succesvol; de webserver heeft een document teruggestuurd, je vindt dat onder `r.text`. We laten hier de eerste 500 tekens zien:

In [174]:
r.text[0:500]

'\n<!DOCTYPE html>\n<html lang="en">\n<!-- Generated from TermPageEx.j2 -->\n    \n    \n    <head>\n\n    <title>email - Schema.org Property</title>\n    <meta charset="utf-8" >\n    <meta name="viewport" content="width=device-width, initial-scale=1">\n    <meta name="description" content="Schema.org Property: email - Email address." />\n    <link rel="shortcut icon" type="image/png" href="/docs/favicon.ico"/>\n    <link rel="stylesheet" type="text/css" href="/docs/schemaorg.css" />\n    <link rel="stylesheet'

Dit is hetzelfde HTML-document dat je krijgt als je die link in de browser opent.
Voor computers is dit document niet erg handig, maar er zit een JSON-LD document "verstopt" in de HTML-code.

Via de Python-library BeautifulSoup (REF), voor het verwerken van HTML-documenten, kun je die gegevens vinden, als een script met als type: "application/ld+json"):

In [176]:
soup =  BeautifulSoup(r.text, "html.parser")
json_script = soup.find(type="application/ld+json")
json_script

<script type="application/ld+json">
{
  "@context": {
    "rdf": "http://www.w3.org/1999/02/22-rdf-syntax-ns#",
    "rdfs": "http://www.w3.org/2000/01/rdf-schema#",
    "schema": "http://schema.org/",
    "xsd": "http://www.w3.org/2001/XMLSchema#"
  },
  "@id": "schema:email",
  "@type": "rdf:Property",
  "rdfs:comment": "Email address.",
  "rdfs:label": "email",
  "schema:domainIncludes": [
    {
      "@id": "schema:Organization"
    },
    {
      "@id": "schema:ContactPoint"
    },
    {
      "@id": "schema:Person"
    }
  ],
  "schema:rangeIncludes": {
    "@id": "schema:Text"
  }
}
</script>

Het JSON-LD object in dit script krijg je via:

In [177]:
json.loads(json_script.contents[0].string)

{'@context': {'rdf': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#',
  'rdfs': 'http://www.w3.org/2000/01/rdf-schema#',
  'schema': 'http://schema.org/',
  'xsd': 'http://www.w3.org/2001/XMLSchema#'},
 '@id': 'schema:email',
 '@type': 'rdf:Property',
 'rdfs:comment': 'Email address.',
 'rdfs:label': 'email',
 'schema:domainIncludes': [{'@id': 'schema:Organization'},
  {'@id': 'schema:ContactPoint'},
  {'@id': 'schema:Person'}],
 'schema:rangeIncludes': {'@id': 'schema:Text'}}

**Opdracht**

* vergelijk het JSON-LD object met de gegevens die je via de browser vindt op [http://schema.org/email](http://schema.org/email).
* wat is het type van de waarde van het veld [http://schema.org/email](http://schema.org/email)?
* in welk soort documenten kan het veld [http://schema.org/email](http://schema.org/email) voorkomen?
* wat is het "label" bij [http://schema.org/email](http://schema.org/email)?
* wat is de betekenis van "label" volgens het bijbehorende schema?

### Functie get_json_ld

We definiëren een functie om een dergelijk JSON-LD script uit het HTML-bestand bij een gegeven URL te vinden, als dit er is. Het resultaat None geeft aan dat we bij die URL geen JSON-LD object kunnen vinden.

In [ ]:
def get_json_ld (url):
    r = requests.get(url)
    if r.status_code != 200: 
        return None
    soup =  BeautifulSoup(r.text, "html.parser")
    json_script = soup.find(type="application/ld+json")
    if json_script == None:
        return None
    return json.loads(json_script.contents[0].string)

**Opdrachten**

* zoek de gegevens op van [http://schema.org/productionDate](http://schema.org/productionDate)
    * via de browser
    * via de functie `get_json_ld`
* zoek de gegevens op van [http://schema.org/Person](http://schema.org/Person)
    * via de browser
    * via de functie `get_json_ld`

## (4) Gebruik links in de gegevens bij een URL

In het web gebruik je links om de verschillende documenten aan elkaar te koppelen.
In het geval van Linked Data kun je dat ook in de data zelf doen.
Dit heb je al gezien voor de gegevens in de schema's.
Je kunt het ook voor de waarden zelf doen.

We geven een aantal voorbeelden van het gebruik van Linked Data.
Ga zelf na hoe daarin links naar andere data gebruikt worden.

## JSON-LD voor zoekmachines

Met behulp van gestructureerde gegevens kunnen zoekmachines veel preciezer zoeken,
en zoekresultaten beter representeren.

Google beschrijft in de [Google structured data richtlijnen](https://developers.google.com/search/docs/guides/intro-structured-data) hoe je deze json-lddata in je webpagina's kunt opnemen.

We geven hieronder enkele voorbeelden van websites met ingebedde json-ld gegevens

In [179]:
get_json_ld("https://ieni.org")

{'@context': 'http://schema.org',
 '@type': 'Organization',
 'url': 'https://ieni.org/',
 'sameAs': ['https://twitter.com/@ieni'],
 '@id': '#organization',
 'name': 'Vakvereniging i&amp;i',
 'logo': 'https://ieni.org/assets/img/brand/header-dark.svg'}

In [181]:
get_json_ld("https://www.smulweb.nl/recepten/941502/Gekookt-ei")

{'@context': 'https://schema.org/',
 '@type': 'Recipe',
 'keywords': 'Recept, Gekookt ei, Diner, Nederlandse keuken, Zacht en romig',
 'author': {'@type': 'Person', 'name': 'zolar'},
 'name': 'Gekookt ei',
 'recipeCategory': 'Feestmaaltijd',
 'recipeCuisine': 'Nederlands',
 'cookTime': 'PT10M',
 'totalTime': 'PT10M',
 'recipeIngredient': 'ei',
 'recipeInstructions': 'Kook het ei.',
 'description': 'Gekookt ei Kook het ei.',
 'recipeYield': '1 personen',
 'image': 'https://images.smulweb.nl/sw3/sw_fallback.png',
 'aggregateRating': {'@type': 'AggregateRating',
  'ratingValue': '5.0',
  'bestRating': '5',
  'worstRating': '0',
  'ratingCount': '4'},
 'comment': [{'@type': 'Comment',
   'text': 'Geweldig recept! Misschien eerst proberen voordat ik hey met Pasen ga serveren :-)',
   'datePublished': '16-03-2016',
   'author': {'@type': 'Person', 'name': 'Harry66'}},
  {'@type': 'Comment',
   'text': 'Je moet van tevoren kijken of het niet al gebakken is........ Maar dat kan toch niet in de

In [182]:
get_json_ld("https://nl.wikipedia.org/wiki/Hedy_Lamarr")

{'@context': 'https://schema.org',
 '@type': 'Article',
 'name': 'Hedy Lamarr',
 'url': 'https://nl.wikipedia.org/wiki/Hedy_Lamarr',
 'sameAs': 'http://www.wikidata.org/entity/Q49034',
 'mainEntity': 'http://www.wikidata.org/entity/Q49034',
 'author': {'@type': 'Organization',
  'name': 'Bijdragers aan Wikimedia projecten'},
 'publisher': {'@type': 'Organization',
  'name': 'Wikimedia Foundation, Inc.',
  'logo': {'@type': 'ImageObject',
   'url': 'https://www.wikimedia.org/static/images/wmf-hor-googpub.png'}},
 'datePublished': '2005-03-15T19:42:44Z',
 'dateModified': '2020-11-09T12:29:00Z',
 'image': 'https://upload.wikimedia.org/wikipedia/commons/6/61/Hedy_lamarr_-_1940.jpg',
 'headline': 'Oostenrijks-Amerikaans actrice en uitvindster'}

## Het web van linked data

Zoals het web een web van documenten vormt, geeft linked data een web van data.
De [Linked Open Data Cloud](https://lod-cloud.net) is daarvan een deel met vrij toegankelijke en herbruikbare ("open") data.

* Wikipedia bevat naast de tekstuele data, ook gestructureerde data 
* [Wikidata](https://wikidata.org) is de open data-wiki van Mediawiki (Wikipedia)
* [DBpedia](https://dbpedia.org) is een open data-wiki op basis van de gegevens in Wikipedia en andere bronnen.

Je kunt deze gegevens op verschillende manieren benaderen, bijvoorbeeld via SPARQL queries.

*Opmerking*: deze gegevens zijn bedoeld om door computers gelezen te worden; voor mensen zijn de resultaten niet altijd handig.

Deze linked data vormt de eerste stap naar het Semantische web.

**Opdrachten**

* zoek Hedy Lamarr op Wikidata
* zoek Hedy Lamarr op DBPedia
** gebruik bijvoorbeeld: http://dbpedia.org/fct/facet.vsp
** wat is haar rol in "How we got to now"?

-----

In [2]:
# productionDate
# Person

headers = {"Content-Type": "application/ld_json"}

r = requests.get("https://schema.org/docs/jsonldcontext.jsonld", headers=headers)
r.status_code

200

In [3]:
for head in r.headers:
    print(head,": ", r.headers[head])

Access-Control-Allow-Credentials :  true
Access-Control-Allow-Headers :  Accept
Access-Control-Allow-Methods :  GET
Access-Control-Allow-Origin :  *
Date :  Sun, 06 Dec 2020 08:53:17 GMT
Expires :  Sun, 06 Dec 2020 09:03:17 GMT
ETag :  "QE7EVQ"
X-Cloud-Trace-Context :  7b5076407dc9b1496396e66db1a5d1b8
Content-Type :  application/ld+json
Server :  Google Frontend
Content-Length :  156492
Age :  593
Cache-Control :  public, max-age=600
Alt-Svc :  h3-29=":443"; ma=2592000,h3-T051=":443"; ma=2592000,h3-Q050=":443"; ma=2592000,h3-Q046=":443"; ma=2592000,h3-Q043=":443"; ma=2592000,quic=":443"; ma=2592000; v="46,43"


In [7]:
r.text[0:500]

'{\n  "@context": {\n        "type": "@type",\n        "id": "@id",\n        "HTML": { "@id": "rdf:HTML" },\n\n        "@vocab": "http://schema.org/",\n        "rdf": "http://www.w3.org/1999/02/22-rdf-syntax-ns#",\n        "rdfs": "http://www.w3.org/2000/01/rdf-schema#",\n        "xsd": "http://www.w3.org/2001/XMLSchema#",\n        "schema": "http://schema.org/",\n        "owl": "http://www.w3.org/2002/07/owl#",\n        "dc": "http://purl.org/dc/elements/1.1/",\n        "dct": "http://purl.org/dc/terms/",\n  '

In [26]:
r.json()["@context"]["givenName"]

{'@id': 'schema:givenName'}

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
soup = BeautifulSoup(r.text, "html.parser")

In [ ]:
soup.find_all("link")

In [ ]:
scripts = soup.find_all('script')

In [ ]:
for s in scripts:
    print(s.attrs.keys())
    if "type" in s.attrs.keys():
        print(s["type"])
    print(s)

In [ ]:
xx = {"aap": 100}
print(xx.keys())
if 'aap' in xx.keys():
    print('yes')

In [ ]:
json_string = soup.find(type="application/ld+json").contents[0].string

In [ ]:
obj = json.loads(json_string)
obj

In [ ]:
obj["@id"]

In [ ]:
obj["@graph"]["@type"]

In [ ]:
obj["rdfs:label"]

In [ ]:
obj["rdfs:comment"]

In [ ]:
obj.keys()

In [ ]:
graph = obj["@graph"]

Ik vind het lastig om het schema van Person te vinden, zoals dat gebruikt wordt in de voorbeelden.

In de voorbeelden is er soms sprake van een *graph* van meerdere objecten/documenten, bijvoorbeeld van een boek en de vertaling daarvan, of van een boek en de auteur. Voor een webpagina is dat te begrijpen, voor een document in een database is dat wat lastig (dit lijkt een vorm van embedding, die je waarschijnlijk in een database niet zo zult gebruiken).

In [ ]:
for elt in graph:
    print(elt["@id"], " ", elt["@type"])
    if elt["@type"] == "rdfs:Class":
        print(elt)

In [ ]:
type(graph)

In [22]:
person = {
  "@context": "https://schema.org/docs/jsonldcontext.jsonld",
  "@type": "Person",
  "address": {
    "@type": "PostalAddress",
    "addressLocality": "Seattle",
    "addressRegion": "WA",
    "postalCode": "98052",
    "streetAddress": "20341 Whitworth Institute 405 N. Whitworth"
  },
  "colleague": [
    "http://www.xyz.edu/students/alicejones.html",
    "http://www.xyz.edu/students/bobsmith.html"
  ],
  "email": ["mailto:jane-doe@xyz.edu", "jane@ziggo.nl"],
  "image": "janedoe.jpg",
  "jobTitle": "Professor",
  "name": "Jane Doe",
  "givenName": "Jane",  
  "telephone": "(425) 123-4567",
  "url": "http://www.janedoe.com"
}

In [23]:
from pyld import jsonld
import json

expanded = jsonld.expand(person)
expanded

[{'@type': ['http://schema.org/Person'],
  'http://schema.org/address': [{'@type': ['http://schema.org/PostalAddress'],
    'http://schema.org/addressLocality': [{'@value': 'Seattle'}],
    'http://schema.org/addressRegion': [{'@value': 'WA'}],
    'http://schema.org/postalCode': [{'@value': '98052'}],
    'http://schema.org/streetAddress': [{'@value': '20341 Whitworth Institute 405 N. Whitworth'}]}],
  'http://schema.org/colleague': [{'@id': 'http://www.xyz.edu/students/alicejones.html'},
   {'@id': 'http://www.xyz.edu/students/bobsmith.html'}],
  'http://schema.org/email': [{'@value': 'mailto:jane-doe@xyz.edu'},
   {'@value': 'jane@ziggo.nl'}],
  'http://schema.org/givenName': [{'@value': 'Jane'}],
  'http://schema.org/image': [{'@id': 'janedoe.jpg'}],
  'http://schema.org/jobTitle': [{'@value': 'Professor'}],
  'http://schema.org/name': [{'@value': 'Jane Doe'}],
  'http://schema.org/telephone': [{'@value': '(425) 123-4567'}],
  'http://schema.org/url': [{'@id': 'http://www.janedoe.c

De expansie van het voorbeeld in de JSON-LD "playground" gaat wel goed (https://json-ld.org/playground/); met de pyld-library lukt het nog niet zonder meer.

> Zie de fout in: pyld - https://github.com/digitalbazaar/pyld/issues/128. Er is kennelijk een verbetering onderweg.
Als we in plaats van schema.org gebruiken: "https://schema.org/docs/jsonldcontext.jsonld", gaat dit wel goed.

In [ ]:
r1 = requests.get("https://schema.org/version/latest/schemaorg-current-http.jsonld")
r1.status_code

In [ ]:
schemas = r1.json()

In [ ]:
schemas

## JSON-LD

In een JSON-object kun je namen gebruiken zoals "voornaam" en "adres".
Voor de menselijke lezer is vaak wel (ongeveer) duidelijk wat deze betekenen.
Maar bij ingewikkelder schema's is dat vaak minder duidelijk.

Door het gebruik van een schema voor een collection kun je ervoor zorgen dat de documenten in die collection in elk geval dezelfde namen gebruiken voor dezelfde begrippen. Dus bijvoorbeeld allemaal "givenName" in plaats van "firstName".

Als je de standaard-schema's van schema.org gebruikt als uitgangspunt weet je dat je aansluit op naamgeving die elders gebruikt wordt.

Met behulp van JSON-LD kun je dit nog een stap verder brengen: je koppelt de namen die je gebruikt direct aan de namen in (bijvoorbeeld) schema.org.

De namen van schema.org kun je toevoegen als *context* aan je JSON-objecten: de veldnaam `givenName` komt dan overeen met [http://schema.org/givenName](http://schema.org/givenName). Met andere woorden: een naam komt overeen met een (unieke) URL. Dit is één van de principes van het web: je gebruikt URIs (URLs) voor het *identificeren* van dingen ("resources") op een eenduidige manier. Omdat de naamgeving in het web wereldwijd dezelfde is, is voor iedereen duidelijk wat hier bedoeld wordt.

Via een context kun je een ook een lokale naam invoeren voor een begrip dat via een URL gedefinieerd is; bijvoorbeeld:

```
"schema": "http://schema.org/",
"voornaam": {"@id": "schema:givenName"}
```

Dit is een afkorting voor:

```
"voornaam": {"@id": "http://schema.org/givenName"}
```

Op deze manier kun je je eigen namen gebruiken, in de lokale taal, zonder de unieke identificatie van schema.org te verliezen.

> Dit maakt het gemakkelijker om aan te sluiten bij lokale gewoontes.

De namen van schema.org vormen niet alleen een unieke identificatie: via de URL kun je ook informatie over een naam ophalen. In de eerste plaats als gewoon HTML document, geschikt voor de menselijke lezer, maar daarnaast ook in een vorm die beter door machines verwerkt kan worden, in `json-ld` formaat. Deze beschrijving is te vinden in een apart script-gedeelte in het HTML-bestand.



## De betekenis van een naam in json-ld formaat

De beschrijving van `http://schema.org/givenName` is "verstopt" in het bijbehorende HTML-document, in de vorm van een script in json-ld formaat.

Met behulp van de (handige!) library `BeautifulSoup` voor het zoeken in een HTML-pagina kunnen we dit script vinden.
We moeten eerst het HTML-documen laden, via de `requests`-library:

In [27]:
r = requests.get("https://schema.org/givenName")
r.status_code

200

Als `r.status_code` gelijk is aan 200 (OK), dan bevat `r.text` de HTML-tekst van de response:

In [29]:
r.text[0:500]

'\n<!DOCTYPE html>\n<html lang="en">\n<!-- Generated from TermPageEx.j2 -->\n    \n    \n    <head>\n\n    <title>givenName - Schema.org Property</title>\n    <meta charset="utf-8" >\n    <meta name="viewport" content="width=device-width, initial-scale=1">\n    <meta name="description" content="Schema.org Property: givenName - Given name. In the U.S., the first name of a Person." />\n    <link rel="shortcut icon" type="image/png" href="/docs/favicon.ico"/>\n    <link rel="stylesheet" type="text/css" href="/do'

Dit document verwerken we via `BeautifulSoup`; we zoeken naar het script met als type: `application/ld_json`

In [32]:
soup = BeautifulSoup(r.text, "html.parser")

In [35]:
json_script = soup.find(type="application/ld+json")
json_script

<script type="application/ld+json">
{
  "@context": {
    "rdf": "http://www.w3.org/1999/02/22-rdf-syntax-ns#",
    "rdfs": "http://www.w3.org/2000/01/rdf-schema#",
    "schema": "http://schema.org/",
    "xsd": "http://www.w3.org/2001/XMLSchema#"
  },
  "@id": "schema:givenName",
  "@type": "rdf:Property",
  "rdfs:comment": "Given name. In the U.S., the first name of a Person.",
  "rdfs:label": "givenName",
  "schema:domainIncludes": {
    "@id": "schema:Person"
  },
  "schema:rangeIncludes": {
    "@id": "schema:Text"
  }
}
</script>

Van dit script hebben we de inhoud nodig; deze moeten we van string-formaat (JSON) omzetten naar een Python-object:

In [36]:
description = json.loads(json_script.contents[0].string)
description

{'@context': {'rdf': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#',
  'rdfs': 'http://www.w3.org/2000/01/rdf-schema#',
  'schema': 'http://schema.org/',
  'xsd': 'http://www.w3.org/2001/XMLSchema#'},
 '@id': 'schema:givenName',
 '@type': 'rdf:Property',
 'rdfs:comment': 'Given name. In the U.S., the first name of a Person.',
 'rdfs:label': 'givenName',
 'schema:domainIncludes': {'@id': 'schema:Person'},
 'schema:rangeIncludes': {'@id': 'schema:Text'}}

In [33]:
json_string = soup.find(type="application/ld+json").contents[0].string
json_string

'\n{\n  "@context": {\n    "rdf": "http://www.w3.org/1999/02/22-rdf-syntax-ns#",\n    "rdfs": "http://www.w3.org/2000/01/rdf-schema#",\n    "schema": "http://schema.org/",\n    "xsd": "http://www.w3.org/2001/XMLSchema#"\n  },\n  "@id": "schema:givenName",\n  "@type": "rdf:Property",\n  "rdfs:comment": "Given name. In the U.S., the first name of a Person.",\n  "rdfs:label": "givenName",\n  "schema:domainIncludes": {\n    "@id": "schema:Person"\n  },\n  "schema:rangeIncludes": {\n    "@id": "schema:Text"\n  }\n}\n'

In [34]:
json.loads(json_string)

{'@context': {'rdf': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#',
  'rdfs': 'http://www.w3.org/2000/01/rdf-schema#',
  'schema': 'http://schema.org/',
  'xsd': 'http://www.w3.org/2001/XMLSchema#'},
 '@id': 'schema:givenName',
 '@type': 'rdf:Property',
 'rdfs:comment': 'Given name. In the U.S., the first name of a Person.',
 'rdfs:label': 'givenName',
 'schema:domainIncludes': {'@id': 'schema:Person'},
 'schema:rangeIncludes': {'@id': 'schema:Text'}}

In [43]:
r1 = requests.get("https://nl.wikipedia.org/wiki/Philippe_Herreweghe")
r1.status_code

200

In [48]:
r1.text[0:500]

'<!DOCTYPE html>\n<html class="client-nojs" lang="nl" dir="ltr">\n<head>\n<meta charset="UTF-8"/>\n<title>Philippe Herreweghe - Wikipedia</title>\n<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":[",\\t.",".\\t,"],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","januari","februari","maart","april","mei","juni","juli","augustus","september","oktober","november","december"],"wgRequestId":"2cfd3b15-ba1b-4148-b6fe-d3'

In [45]:
soup1 =  BeautifulSoup(r1.text, "html.parser")

In [46]:
json_script1 = soup1.find(type="application/ld+json")
json_script1

<script type="application/ld+json">{"@context":"https:\/\/schema.org","@type":"Article","name":"Philippe Herreweghe","url":"https:\/\/nl.wikipedia.org\/wiki\/Philippe_Herreweghe","sameAs":"http:\/\/www.wikidata.org\/entity\/Q168039","mainEntity":"http:\/\/www.wikidata.org\/entity\/Q168039","author":{"@type":"Organization","name":"Bijdragers aan Wikimedia projecten"},"publisher":{"@type":"Organization","name":"Wikimedia Foundation, Inc.","logo":{"@type":"ImageObject","url":"https:\/\/www.wikimedia.org\/static\/images\/wmf-hor-googpub.png"}},"datePublished":"2005-07-14T20:16:19Z","dateModified":"2019-09-09T03:08:28Z","image":"https:\/\/upload.wikimedia.org\/wikipedia\/commons\/3\/38\/Herreweghe675.jpg","headline":"Belgisch dirigent"}</script>

Op de website van de Chinees in Waalre worden wel de namen van schema.org gebruikt, maar (nog) niet de JSON-LD notatie; Google stelt dat deze tegenwoordig de voorkeur verdient.

In [47]:
description1 = json.loads(json_script1.contents[0].string)
description1

{'@context': 'https://schema.org',
 '@type': 'Article',
 'name': 'Philippe Herreweghe',
 'url': 'https://nl.wikipedia.org/wiki/Philippe_Herreweghe',
 'sameAs': 'http://www.wikidata.org/entity/Q168039',
 'mainEntity': 'http://www.wikidata.org/entity/Q168039',
 'author': {'@type': 'Organization',
  'name': 'Bijdragers aan Wikimedia projecten'},
 'publisher': {'@type': 'Organization',
  'name': 'Wikimedia Foundation, Inc.',
  'logo': {'@type': 'ImageObject',
   'url': 'https://www.wikimedia.org/static/images/wmf-hor-googpub.png'}},
 'datePublished': '2005-07-14T20:16:19Z',
 'dateModified': '2019-09-09T03:08:28Z',
 'image': 'https://upload.wikimedia.org/wikipedia/commons/3/38/Herreweghe675.jpg',
 'headline': 'Belgisch dirigent'}

In [63]:
r2 = requests.get("https://www.ah.nl/winkel/albert-heijn/waalre/den-hof/8561")
r2.status_code

200

In [64]:
r2.text[0:5000]

'<!doctype html>\n    <html  lang="nl_NL">\n        <head>\n            <meta http-equiv="X-UA-Compatible" content="IE=edge">\n            <meta name="csrf-token" content="bglmEgb3-eQI1NgrcsZyhoBVQZx9oceFYaZ8">\n            <title data-react-helmet="true">AH Den Hof openingstijden |Albert Heijn</title>\n            <meta data-react-helmet="true" name="settings:cookie-version" content="1.6"/><meta data-react-helmet="true" name="dynatrace-beacon" content=""/><meta data-react-helmet="true" name="theme-color" content="#2f7fb5"/><meta data-react-helmet="true" name="msapplication-TileColor" content="#2f7fb5"/><meta data-react-helmet="true" name="msapplication-TileImage" content="/winkel/assets/08c0d513.png"/><meta data-react-helmet="true" name="viewport" content="width=device-width, initial-scale=1.0"/><meta data-react-helmet="true" name="app:id" content="ah-stores"/><meta data-react-helmet="true" name="app:version" content="0.207.0"/><meta data-react-helmet="true" name="settings:cookie-vers

In [65]:
soup2 =  BeautifulSoup(r2.text, "html.parser")

In [66]:
json_script2 = soup2.find(type="application/ld+json")
json_script2

<script data-react-helmet="true" type="application/ld+json">{"url":"https://www.ah.nl/winkel/albert-heijn/waalre/den-hof/8561","@context":"https://schema.org","@type":"GroceryStore","@id":"https://www.ah.nl/winkel/albert-heijn/waalre/den-hof/8561","name":"AH Den Hof","image":"https://www.ah.nl//winkel/assets/9cc2b5e4.svg","address":{"@type":"PostalAddress","streetAddress":"Den Hof 4","addressLocality":"Waalre","postalCode":"5582JZ","addressCountry":"NLD"},"geo":{"@type":"GeoCoordinates","latitude":51.395218,"longitude":5.474604},"telephone":"0402212354","openingHoursSpecification":[{"@type":"OpeningHoursSpecification","dayOfWeek":"Monday","opens":"07:00","closes":"21:00"},{"@type":"OpeningHoursSpecification","dayOfWeek":"Tuesday","opens":"07:00","closes":"21:00"},{"@type":"OpeningHoursSpecification","dayOfWeek":"Wednesday","opens":"07:00","closes":"21:00"},{"@type":"OpeningHoursSpecification","dayOfWeek":"Thursday","opens":"07:00","closes":"21:00"},{"@type":"OpeningHoursSpecification",

In [ ]:
json_script2 = soup2.find(type="application/ld+json")
json_script2

In [67]:
description2 = json.loads(json_script2.contents[0].string)
description2

{'url': 'https://www.ah.nl/winkel/albert-heijn/waalre/den-hof/8561',
 '@context': 'https://schema.org',
 '@type': 'GroceryStore',
 '@id': 'https://www.ah.nl/winkel/albert-heijn/waalre/den-hof/8561',
 'name': 'AH Den Hof',
 'image': 'https://www.ah.nl//winkel/assets/9cc2b5e4.svg',
 'address': {'@type': 'PostalAddress',
  'streetAddress': 'Den Hof 4',
  'addressLocality': 'Waalre',
  'postalCode': '5582JZ',
  'addressCountry': 'NLD'},
 'geo': {'@type': 'GeoCoordinates',
  'latitude': 51.395218,
  'longitude': 5.474604},
 'telephone': '0402212354',
 'openingHoursSpecification': [{'@type': 'OpeningHoursSpecification',
   'dayOfWeek': 'Monday',
   'opens': '07:00',
   'closes': '21:00'},
  {'@type': 'OpeningHoursSpecification',
   'dayOfWeek': 'Tuesday',
   'opens': '07:00',
   'closes': '21:00'},
  {'@type': 'OpeningHoursSpecification',
   'dayOfWeek': 'Wednesday',
   'opens': '07:00',
   'closes': '21:00'},
  {'@type': 'OpeningHoursSpecification',
   'dayOfWeek': 'Thursday',
   'opens': '

Met andere woorden: de gegevens van wikidata staan (nog?) niet in JSON-LD formaat

En die van AH wel...

In [87]:
hans = {
  "@context": {
    "@context": "https://schema.org/docs/jsonldcontext.jsonld",
    "voornaam": "givenName"
  },
  "@type": "Person",
  "givenName": "Hans",
  "familyName": "de Boer",
  "email": "hans34@kpnmail.nl"
}

In [88]:
hans_ex = jsonld.expand(hans)
hans_ex

[{'@type': ['http://schema.org/Person'],
  'http://schema.org/email': [{'@value': 'hans34@kpnmail.nl'}],
  'http://schema.org/familyName': [{'@value': 'de Boer'}],
  'http://schema.org/givenName': [{'@value': 'Hans'}]}]

In [86]:
jsonld.compact(hans_ex, "https://schema.org/docs/jsonldcontext.jsonld" )

{'@context': 'https://schema.org/docs/jsonldcontext.jsonld',
 'type': 'Person',
 'email': 'hans34@kpnmail.nl',
 'familyName': 'de Boer',
 'givenName': 'Hans'}

NB: in Python speelt de volgorde van de namen in een dictionary, bij mijn weten, geen rol.
zie het voorbeeld hieronder. (Ik weet niet hoe dat in de MongoDB is.)

Ik begrijp dat je met behulp van een "frame" de velden wel in de gewenste volgorde kunt krijgen.

In [79]:
x = {'aap': 10, 'noot': 'mies', 'zus': 12}
y = {'noot': 'mies', 'zus': 12,'aap': 10}
print(x == y)

True


In [116]:
r4 = requests.get("https://www.lotusgardenwaalre.nl")
r4.status_code

200

In [117]:
soup4 =  BeautifulSoup(r4.text, "html.parser")

In [122]:
json_script4 = soup4.find(type="application/ld+json")
json_script4 == None

True

In [115]:
description4 = json.loads(json_script4.contents[0].string)
description4

{'@context': 'https://schema.org',
 '@graph': [{'@type': 'WebSite',
   '@id': 'https://www.jumbo.eu/#website',
   'url': 'https://www.jumbo.eu/',
   'name': 'Jumbo',
   'description': 'Spellen, puzzles en speelgoed',
   'potentialAction': [{'@type': 'SearchAction',
     'target': 'https://www.jumbo.eu/?s={search_term_string}',
     'query-input': 'required name=search_term_string'}],
   'inLanguage': 'nl'},
  {'@type': 'WebPage',
   '@id': 'https://www.jumbo.eu/#webpage',
   'url': 'https://www.jumbo.eu/',
   'name': 'Home - Jumbo',
   'isPartOf': {'@id': 'https://www.jumbo.eu/#website'},
   'datePublished': '2017-11-07T11:35:51+00:00',
   'dateModified': '2020-11-12T10:22:17+00:00',
   'inLanguage': 'nl',
   'potentialAction': [{'@type': 'ReadAction',
     'target': ['https://www.jumbo.eu/']}]}]}

In [126]:
def get_json_ld (url):
    r = requests.get(url)
    if r.status_code != 200: 
        return None
    soup =  BeautifulSoup(r.text, "html.parser")
    json_script = soup.find(type="application/ld+json")
    if json_script == None:
        return None
    return json.loads(json_script.contents[0].string)

In [127]:
get_json_ld("https://ieni.org")

{'@context': 'http://schema.org',
 '@type': 'Organization',
 'url': 'https://ieni.org/',
 'sameAs': ['https://twitter.com/@ieni'],
 '@id': '#organization',
 'name': 'Vakvereniging i&amp;i',
 'logo': 'https://ieni.org/assets/img/brand/header-dark.svg'}

In [128]:
get_json_ld("https://nl.wikipedia.org/wiki/Hedy_Lamarr")

{'@context': 'https://schema.org',
 '@type': 'Article',
 'name': 'Hedy Lamarr',
 'url': 'https://nl.wikipedia.org/wiki/Hedy_Lamarr',
 'sameAs': 'http://www.wikidata.org/entity/Q49034',
 'mainEntity': 'http://www.wikidata.org/entity/Q49034',
 'author': {'@type': 'Organization',
  'name': 'Bijdragers aan Wikimedia projecten'},
 'publisher': {'@type': 'Organization',
  'name': 'Wikimedia Foundation, Inc.',
  'logo': {'@type': 'ImageObject',
   'url': 'https://www.wikimedia.org/static/images/wmf-hor-googpub.png'}},
 'datePublished': '2005-03-15T19:42:44Z',
 'dateModified': '2020-11-09T12:29:00Z',
 'image': 'https://upload.wikimedia.org/wikipedia/commons/6/61/Hedy_lamarr_-_1940.jpg',
 'headline': 'Oostenrijks-Amerikaans actrice en uitvindster'}

* volg de link naar wikidata voor meer gestructureerde gegevens over Hedy LLamar

In [144]:
get_json_ld('http://dbpedia.org/page/Hedy_Lamarr')

IndexError: list index out of range

In [150]:
url = 'http://dbpedia.org?resource%2FHedy_Lamarr&format=application%2Fjson-ld'
r = requests.get(url)
if r.status_code == 200:
    
    soup =  BeautifulSoup(r.text, "html.parser")
    json_script = soup.find(type="application/ld+json")
json_script    

In [139]:
r10 = requests.get('http://www.wikidata.org/entity/Q49034')
r10.json()["entities"]["Q49034"]["descriptions"]["nl"]

{'language': 'nl', 'value': 'Oostenrijks-Amerikaans actrice en uitvindster'}

In [151]:
r10.json()["entities"]["Q49034"].

SyntaxError: invalid syntax (<ipython-input-151-516ceb22f86d>, line 1)

In [154]:
get_json_ld("https://www.smulweb.nl/recepten/1064577/Weihnachtsstolle-of-duitse-kerststol")

{'@context': 'https://schema.org/',
 '@type': 'Recipe',
 'keywords': 'Recept, Weihnachtsstolle of duitse kerststol, Ontbijt, Duitse keuken, Zoet',
 'author': {'@type': 'Person', 'name': 'pwiggers'},
 'name': 'Weihnachtsstolle of duitse kerststol',
 'recipeCategory': 'Brood',
 'recipeCuisine': 'Duits',
 'cookTime': 'PT1H30M',
 'totalTime': 'PT1H30M',
 'recipeIngredient': '500 gr bloem - 1 zakje bakpoeder - 180 gr suiker - 1 zakje vanillesuiker - mespunt zout - 4 druppels amandelaroma - 5 druppels citroenaroma - 1 borrelglas rum - 2 eieren - 120 gr koude boter - 250 gr magere kwark - 200 gr gewassen rozijnen - 100 gr gewassen krenten - 150 gr gemalen amandelen - 50 gr gedroogde sinaasappelschilletjes - 50 gr gesnipperde sukade\nOm te bestrijken: 50 gram zachte boter\nOm te bestrooien: 50 gram poedersuiker',
 'recipeInstructions': 'Voorbereiding\n\nRozijnen, krenten, sukade en sinaasappelschilletjes avond tevoren in de rum zetten. Is de kwark nat dan even laten uitlekken in een zeef.\nBlo